In [1]:
import polars as pl
import tarfile
import os
import datetime as dt
from pathlib import Path
from tqdm import tqdm

from src.preprocessing import preprocess_all_tickers, select_complete_tickers, count_tickers_with_expected_rows

# Autorealod extension for Jupyter Notebooks
%load_ext autoreload
%autoreload 2

In [2]:
preprocess_all_tickers()

2025-11-25 17:47:05,609 | INFO | Starting preprocessing of all tickers...
2025-11-25 17:47:05,610 | INFO | Scanning RAW_FOLDER for tickers...
2025-11-25 17:47:05,662 | INFO | Found 102 tickers.
2025-11-25 17:47:05,666 | INFO | 9 tickers need preprocessing.


Preprocessing tickers:   0%|          | 0/9 [00:00<?, ?it/s]

2025-11-25 17:47:05,714 | INFO | === Preprocessing ticker BRK.B.OQ ===
2025-11-25 17:47:05,714 | INFO | [BRK.B.OQ] Extracting archives...
2025-11-25 17:47:05,734 | INFO | Loading parquet files in data/extracted/SP100/bbo/BRK.B.OQ...
2025-11-25 17:47:05,737 | INFO | Loaded 0 parquet files.
2025-11-25 17:47:05,738 | ERROR | No parquet files found for ticker BRK.B.OQ
2025-11-25 17:47:05,738 | ERROR | [BRK.B.OQ] Error during preprocessing: No parquet files found for ticker BRK.B.OQ
2025-11-25 17:47:05,740 | INFO | === Preprocessing ticker ORCL.OQ ===
2025-11-25 17:47:05,741 | INFO | [ORCL.OQ] Extracting archives...
2025-11-25 17:47:05,758 | INFO | Loading parquet files in data/extracted/SP100/bbo/ORCL.OQ...
2025-11-25 17:47:05,761 | INFO | Loaded 0 parquet files.
2025-11-25 17:47:05,762 | ERROR | No parquet files found for ticker ORCL.OQ
2025-11-25 17:47:05,762 | ERROR | [ORCL.OQ] Error during preprocessing: No parquet files found for ticker ORCL.OQ
2025-11-25 17:47:05,763 | INFO | === Pre

In [3]:
select_complete_tickers(exception_dates=[dt.datetime(2015, 11, 27).date()], delete_exception_days=True)

2025-11-25 17:47:06,067 | INFO | Starting selection of complete tickers...
2025-11-25 17:47:13,803 | WARNING | [PYPL.OQ.parquet] Missing 136 trading days: [datetime.date(2015, 1, 2), datetime.date(2015, 1, 5), datetime.date(2015, 1, 6), datetime.date(2015, 1, 7), datetime.date(2015, 1, 8), datetime.date(2015, 1, 9), datetime.date(2015, 1, 12), datetime.date(2015, 1, 13), datetime.date(2015, 1, 14), datetime.date(2015, 1, 15), datetime.date(2015, 1, 16), datetime.date(2015, 1, 20), datetime.date(2015, 1, 21), datetime.date(2015, 1, 22), datetime.date(2015, 1, 23), datetime.date(2015, 1, 26), datetime.date(2015, 1, 27), datetime.date(2015, 1, 28), datetime.date(2015, 1, 29), datetime.date(2015, 1, 30), datetime.date(2015, 2, 2), datetime.date(2015, 2, 3), datetime.date(2015, 2, 4), datetime.date(2015, 2, 5), datetime.date(2015, 2, 6), datetime.date(2015, 2, 9), datetime.date(2015, 2, 10), datetime.date(2015, 2, 11), datetime.date(2015, 2, 12), datetime.date(2015, 2, 13), datetime.date(20

In [4]:
count_tickers_with_expected_rows()

2025-11-25 17:47:16,130 | INFO | Counting rows for each selected ticker...
2025-11-25 17:47:16,162 | WARNING | MA.N.parquet: 192793 rows (below expected 219785)
2025-11-25 17:47:16,182 | WARNING | GOOG.OQ.parquet: 219361 rows (below expected 219785)
2025-11-25 17:47:16,203 | WARNING | GOOGL.OQ.parquet: 219049 rows (below expected 219785)
2025-11-25 17:47:16,220 | WARNING | PEP.N.parquet: 193066 rows (below expected 219785)
2025-11-25 17:47:16,239 | WARNING | AMGN.OQ.parquet: 219438 rows (below expected 219785)
2025-11-25 17:47:16,255 | WARNING | BK.N.parquet: 192679 rows (below expected 219785)
2025-11-25 17:47:16,268 | WARNING | F.N.parquet: 192741 rows (below expected 219785)
2025-11-25 17:47:16,284 | WARNING | C.N.parquet: 193058 rows (below expected 219785)
2025-11-25 17:47:16,308 | WARNING | CVS.N.parquet: 193102 rows (below expected 219785)
2025-11-25 17:47:16,327 | WARNING | VZ.N.parquet: 195939 rows (below expected 219785)
2025-11-25 17:47:16,354 | WARNING | DIS.N.parquet: 1961

2